# Connecting Python and ANSYS by PARANSYS

In this example the APDL script `BeckCantileverAPDL.inp` is used on ANSYS by PARANSYS. The input variables from the script are `q,l,b,h` and the ouput is the stress on a cantilever beam (`stress`).

## Importing all that will be used:

In [41]:
import paransys
import numpy as np
import pandas as pd
from IPython.display import display

## Setup:
First we need to find the ANSYS executable location (`ansloc`) and define the working directory that ANSYS will use (`workingdir`). I'm using ANSYS 2019 R2 and the working dir is at `C:\Temp\wk`.

In [42]:

ansloc = 'C:\\Program Files\\ANSYS Inc\\v194\\ansys\\bin\\winx64\\ansys194.exe'
workingdir = 'C:\\Temp\\wk'
# Create ans object that is ANSYS
ans = paransys.ANSYS(exec_loc=ansloc, run_location=workingdir, jobname='file', nproc=4, override=True, cleardir=False, add_flags='')
# Activate the output to Python
ans.Info(True)

ANSYS properties defined as:
   Executable file: "C:\Program Files\ANSYS Inc\v194\ansys\bin\winx64\ansys194.exe".
   Working directory: "C:\Temp\wk".
   Jobname: "file".
   Number of processors used: "4".
   Override lock file: "True".
   Clear working directory: "False".
   Additional flags: "".
Now the commands will send a return to Python (like this).


### Set APDL model that will be used:
Here it's at the current directory and have no extra files, so we can do:


In [43]:
ans.SetModel(inputname='BeckCantileverAPDL.inp', extrafiles=[], directory='.')

Input script file and extra files copied to working directory.
   Main APDL script: "BeckCantileverAPDL.inp".
   Extra model files: "[]".
   Input directory: ".".


### Setting parameters that will be setted by Python and the parameters that ANSYS should return:
In this case the input parameters are `q,l,b,h` and the ouput `stress`.


In [44]:
# Set input parameters
ans.CreateVarIn('q')
ans.CreateVarIn('l')
ans.CreateVarIn('b')
ans.CreateVarIn('h')

# Set output parameters
ans.CreateVarOut('stress')

ANSYS input variable "Q" created.
ANSYS input variable "L" created.
ANSYS input variable "B" created.
ANSYS input variable "H" created.
Variable "STRESS" declared as ANSYS output variable.


### Setting analysis lenght:
The number of analysis that will be executed should be defined before setting the values, with this PARANSYS can know if the values are in the righ size.
In this case the lenght is 3 analysis.

In [45]:
ans.SetLength(3)

Analysis lenght set to 3.


The model is now set. 
If you keep the model, variables and lenght that part doesn't need to change anymore. The values could be set as many times as you need, this will be done now.

## Setting vlaues, running and postprocessing:
The 3 analysis that will be done here are used to determine the derivatives of stress in relation to l and h by the finite difference method: $$\frac{\partial stress}{\partial l} \space\space\space and \space\space\space \frac{\partial stress}{\partial h}$$

The values that parameters will assume in the three simualtions are:

| Sim\Var |  q   |   l  |  b  | h   |
|---------|------|------|-----|-----|
| Sim #1  | 1.15 | 60.0 | 4.0 | 1.0 |
| Sim #2  | 1.15 | 62.0 | 4.0 | 1.0 |
| Sim #3  | 1.15 | 60.0 | 4.0 | 1.2 |


In [46]:
# Creatting arrays with the values
# Values at (Sim#1, Sim#2, Sim#3)
q = np.array([1.15,  1.15,  1.15])
l = np.array([60.0,  62.0,  60.0])
b = np.array([4.00,  4.00,  4.00])
h = np.array([1.00,  1.00,  1.20])

# Passing it to ANSYS
ans.SetVarInValues('q', q)
ans.SetVarInValues('l', l)
ans.SetVarInValues('b', b)
ans.SetVarInValues('h', h)

Values of "Q" were set.
Values of "L" were set.
Values of "B" were set.
Values of "H" were set.


### Running:

In [47]:
ans.Run()

Deleting old error log file.
Writing the pdsrun.inp file.
Saving the pdsrun.inp file.
Writing the current.samp file.
Running ANSYS.
Solution is done. It took 0.071076 minutes.


### Acessing the results:
The results will by imported as a dictionary

In [48]:
results = ans.GetVarOutValues()

Importing results from PDS results file ("C:\Temp\wk\file_current.pdrs").


We can print all results using print(), print just `stress` or another whings.

The output parameters always came in UPPER CASE because of ANSYS is made in FORTRAN77 (I think it's because of that!)

In [49]:
# Print all results
print(f'All results: {results}\n')

# Print just stresses
print(f'Just stresses: {results["STRESS"]}\n')

All results: {'ERR': array([0., 0., 0.]), 'STRESS': array([3103.706543, 3314.068848, 2155.351807])}

Just stresses: [3103.706543 3314.068848 2155.351807]



We can make a Pandas DataFrame with all inputs and results!

In [50]:
df = pd.DataFrame({'q': q, 'l': l, 'b': b, 'h': h, 'stress': results['STRESS']})
#print(df) # Console
display(df) # On Jupyter

,q,l,b,h,stress
0,1.15,60.0,4.0,1.0,3103.706543
1,1.15,62.0,4.0,1.0,3314.068848
2,1.15,60.0,4.0,1.2,2155.351807


### Derivatives:
The finite difference method could be described as: $$\frac{\partial g(x)}{\partial x} = \frac{g(x+dh)-g(x)}{dh}$$

The values in the past simulations are:
- First simulation = g(q,l,b,h)
- Second simulation = g(q,l+dh,b,h), being dh=2.0
- Third simulation = g(q,l,b,h+dh), being dh=0.2

So with the first and second lines we can determine $\frac{\partial stress}{\partial l}$ as:

In [51]:
dSdl = (df['stress'][1]-df['stress'][0])/(df['l'][1]-df['l'][0])
print(f'dStress/dl = {dSdl}')

dStress/dl = 105.18115250000005


And with the first and third lines we can determine $\frac{\partial stress}{\partial h}$ as:

In [52]:
dSdh = (df['stress'][2]-df['stress'][0])/(df['h'][2]-df['h'][0])
print(f'dStress/dh = {dSdh}')

dStress/dh = -4741.77368


## Final considerations:
We used PARANSYS to determine the derivatives of an ANSYS model, this is used in PARANSYS.FORM class to apply the First Order Reliability Method, and could be used in this way for many projects (otimization, another reliability problems, Monte Carlo simulations, Neural Networks, etc).

If you have to change the variables or the model of your analysis you may have to use the command `ans.ClearAll()`, this will erase almost everything. If just have to erase the values and change the number of analysis (length) you may use `ans.ClearValues()`, this will keep the model and variables names. 


For now that's all I remember!

Eduardo P. Titello, 2020.